<a href="https://colab.research.google.com/github/manthanthakker/DeepNeuralNetworks/blob/master/Pytorch_CustomNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Pytorch Custom DNN**

Goal: Implement a custom Neural Network to play around with a sample dataset from sci-kit learn. We have selected a sample dataset breast cancer detection which is the most basic classification dataset we could get hold off.


First of all import the necessary modules


In [0]:
from sklearn.datasets import load_breast_cancer
from sklearn import preprocessing
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
import torch
import torchvision.transforms as transforms
import torch
import torch.nn as nn

Now lets load the breast cancer dataset for classification task, which is readily available from scikit learn package. 


> Preprocessing Steps


1.   Normalize the datasets
2.   Split into training and testing
3.   Transform the dataset to Pytorch tensors

You can read more information about the data set by simply reading data.DESCR attribute.

Lets do this





In [0]:
data = load_breast_cancer()
X,y=data.data,data.target

## Normalizing
scalerX = preprocessing.StandardScaler().fit(X)
X=scalerX.transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

print(data.DESCR)

.. _breast_cancer_dataset:

Breast cancer wisconsin (diagnostic) dataset
--------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 569

    :Number of Attributes: 30 numeric, predictive attributes and the class

    :Attribute Information:
        - radius (mean of distances from center to points on the perimeter)
        - texture (standard deviation of gray-scale values)
        - perimeter
        - area
        - smoothness (local variation in radius lengths)
        - compactness (perimeter^2 / area - 1.0)
        - concavity (severity of concave portions of the contour)
        - concave points (number of concave portions of the contour)
        - symmetry 
        - fractal dimension ("coastline approximation" - 1)

        The mean, standard error, and "worst" or largest (mean of the three
        largest values) of these features were computed for each image,
        resulting in 30 features.  For instance, field 3 is Mean Radius, f

So this data set has 30 attributes and a label that the given record has a breast cancer or not. Its a typical Classification problem, so we will be using cross entropy loss function and gradient descent as our training algorithm. 
We will be directly jumping to desinging the neural network as thats what we are trying to learn, one can also explore, random forest, SVM etc. 

This is a typical Neural Network setup, pytorch exposes it with defining classes as interface. All you need to do is extend the nnModule class and leverage all the goodness from the base class to construct your custom Neural Networks. You can also use most of the activation functions and they are part of the nn Module in Pytorch. 

We will be creating a single hidden layer Neural network and would parameterize the hidden neurons, so that we can fine tune it according our experimentation.

Sigmoid activation function is fine. One can use relu too. 

In [0]:
class FeedforwardNeuralNetModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(FeedforwardNeuralNetModel, self).__init__()
        # Linear function
        self.fc1 = nn.Linear(input_dim, hidden_dim) 
        # Non-linearity
        self.sigmoid = nn.Sigmoid()
        # Linear function (readout)
        self.fc2 = nn.Linear(hidden_dim, output_dim)  
    
    def forward(self, x):
        # Linear function  # LINEAR
        out = self.fc1(x)
        # Non-linearity  # NON-LINEAR
        out = self.sigmoid(out)
        # Linear function (readout)  # LINEAR
        out = self.fc2(out)
        return out

In [0]:
print(model.parameters())

print(len(list(model.parameters())))

# FC 1 Parameters 
print(list(model.parameters())[0].size())

# FC 1 Bias Parameters
print(list(model.parameters())[1].size())

# FC 2 Parameters
print(list(model.parameters())[2].size())

# FC 2 Bias Parameters
print(list(model.parameters())[3].size())

<generator object Module.parameters at 0x7fb176883938>
4
torch.Size([8, 30])
torch.Size([8])
torch.Size([2, 8])
torch.Size([2])


Lets train the Neural Network now, it will follow a typical process in any Machine learning algo:



1.   Loading data from numpy as tensors
2.   Clearning the gradients from previous epoch
3.   output=apply the model to the input
4.   compute the loss from ouput and target values
5.   compute the gradients



In [0]:
input_dim = 30
hidden_dim = 8
output_dim = 2

## Create an instance of NN
model = FeedforwardNeuralNetModel(input_dim, hidden_dim, output_dim)

## Select a Loss Function
criterion = nn.CrossEntropyLoss()
learning_rate = 0.15

## Select an optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate,momentum=0.09)  


num_epochs=55000
iter = 0

## Training...
for epoch in range(num_epochs):

        # Load images with gradient accumulation capabilities
        images = torch.from_numpy(X_train).requires_grad_()
        labels=torch.from_numpy(y_train)

        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()
        
        # Forward pass to get output/logits
        outputs = model(images.float())
        
        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)
      
        # Getting gradients w.r.t. parameters
        loss.backward()
       
        # Updating parameters
        optimizer.step()

        iter += 1
        
        if iter % 1000 == 0:
              # Print Loss
            print('Iteration: {}. Loss: {}'.format(iter, loss.item()))

Iteration: 1000. Loss: 0.05288170650601387
Iteration: 2000. Loss: 0.04305471479892731
Iteration: 3000. Loss: 0.036482714116573334
Iteration: 4000. Loss: 0.030957181006669998
Iteration: 5000. Loss: 0.025832319632172585
Iteration: 6000. Loss: 0.020235512405633926
Iteration: 7000. Loss: 0.01587308757007122
Iteration: 8000. Loss: 0.012684253975749016
Iteration: 9000. Loss: 0.010386420413851738
Iteration: 10000. Loss: 0.008670968934893608
Iteration: 11000. Loss: 0.007362376432865858
Iteration: 12000. Loss: 0.006345524452626705
Iteration: 13000. Loss: 0.005540094804018736
Iteration: 14000. Loss: 0.004890663083642721
Iteration: 15000. Loss: 0.004358766600489616
Iteration: 16000. Loss: 0.003917139023542404
Iteration: 17000. Loss: 0.0035460004583001137
Iteration: 18000. Loss: 0.0032307635992765427
Iteration: 19000. Loss: 0.0029604274313896894
Iteration: 20000. Loss: 0.002726608654484153
Iteration: 21000. Loss: 0.002522808965295553
Iteration: 22000. Loss: 0.00234392611309886
Iteration: 23000. Lo

As you can see the accuracy is 96% with just 55000 iterations and it takes hardly 2 minutes to train.. The best model has accuracy of 98% for this model which is also a Neural Network. We can fine tune the network and compare the type1 or type2 errors but is not the scope of this notebook.

In [0]:
with torch.no_grad():
          # Calculate Accuracy   
          
          correct = 0
          total = 0
          # Iterate through test dataset
          test_images = torch.from_numpy(X_test)
          test_target=torch.from_numpy(y_test)

          # Forward pass only to get logits/output
          outputs = model(test_images.float())
          
          #print(outputs.data)
          # # Get predictions from the maximum value
          _, predicted = torch.max(outputs.data, 1)
          

          # Total number of labels
          total += test_target.size(0)
          
          # Total correct predictions
          correct += (predicted == test_target).sum()
          
          accuracy = 100 * correct / total
          print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))

Iteration: 55000. Loss: 0.0006160429329611361. Accuracy: 96
